# Build custom container for Vertex training

In [1]:
!pwd

/home/jupyter/tf_vertex_agents/02-supervised-to-bandit-training


## Load env config

* use the prefix from `00-env-setup`

In [2]:
VERSION        = "v2"                       # TODO
PREFIX         = f'rec-bandits-{VERSION}'   # TODO

print(f"PREFIX: {PREFIX}")

PREFIX: rec-bandits-v2


In [3]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"
VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "rec-bandits-v2"
VERSION                  = "v2"

BUCKET_NAME              = "rec-bandits-v2-hybrid-vertex-bucket"
BUCKET_URI               = "gs://rec-bandits-v2-hybrid-vertex-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://rec-bandits-v2-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

VPC_NETWORK_FULL         = "projects/934903580331/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_NAME    = "mvlens_rec_bandits_v2"
BIGQUERY_TABLE_NAME      = "training_dataset"

REPOSITORY               = "rl-movielens-rec-bandits-v2"

DOCKERNAM

In [4]:
# ! gsutil ls $BUCKET_URI

## imports

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Build Image

In [6]:
# !tree src

## Container Image Variables

In [7]:
# DOCKERNAME = DOCKERNAME_02
# IMAGE_NAME = IMAGE_NAME_02
# IMAGE_URI = IMAGE_URI_02

print(f"DOCKERNAME_02     = {DOCKERNAME_02}")
print(f"REPOSITORY        = {REPOSITORY}")
print(f"IMAGE_NAME_02     = {IMAGE_NAME_02}")
print(f"REMOTE_IMAGE_NAME = {REMOTE_IMAGE_NAME}")
print(f"IMAGE_URI_02      = {IMAGE_URI_02}")

DOCKERNAME_02     = Dockerfile_perarm_feats
REPOSITORY        = rl-movielens-rec-bandits-v2
IMAGE_NAME_02     = train-perarm-feats-v2
REMOTE_IMAGE_NAME = us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/local_docker_tfa
IMAGE_URI_02      = gcr.io/hybrid-vertex/train-perarm-feats-v2


## Create Artifact Repository

If you don't have an existing artifact repository, create one using the gcloud command below

In [8]:
# ! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

## Create Dockerfile

In [9]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/tf_vertex_agents'

## Create train image

* see [example Dockerfile for GPU](https://github.com/GoogleCloudPlatform/cloudml-samples/blob/main/pytorch/containers/quickstart/mnist/Dockerfile-gpu) jobs in Vertex AI
* see deep learning container [example here](https://cloud.google.com/deep-learning-containers/docs/derivative-container), and here for [available DL containers](https://cloud.google.com/deep-learning-containers/docs/choosing-container#versions)
* installing [NVIDIA driver](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.htmlhttps://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html)
* [Tensorflow GPU images and compatiblity](https://www.tensorflow.org/install/source#gpuhttps://www.tensorflow.org/install/source#gpu)
> `tensorflow 2.15` compatible with `CUDA 12.2` and `cuDNN 8.9`

### GPU

In [10]:
# !gcloud container images list --repository="us-docker.pkg.dev/deeplearning-platform-release/gcr.io"

In [11]:
gpu_profiling = True # True | False

print(f"gpu_profiling : {gpu_profiling}")

gpu_profiling : True


In [12]:
if gpu_profiling:
    # TRAIN_BASE_IMAGE = 'tensorflow/tensorflow:2.14.0-gpu'
    # TRAIN_BASE_IMAGE = 'tensorflow/tensorflow:2.15.0-gpu'
    # TRAIN_BASE_IMAGE = "gcr.io/deeplearning-platform-release/tf-gpu.2-15"     # need to test
    TRAIN_BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf-gpu.2-13.py310' # works m111
    NVTOP_RUN = 'RUN apt update && apt -y install nvtop'
    # NVTOP_RUN = 'RUN apt-get update && apt-get -y install nvtop'
else:
    TRAIN_BASE_IMAGE = 'python:3.10'
    NVTOP_RUN = ""
    
RUN_EXPORT = "RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/"
    
print(f"TRAIN_BASE_IMAGE : {TRAIN_BASE_IMAGE}")
print(f"NVTOP_RUN        : {NVTOP_RUN}")
print(f"RUN_EXPORT       : {RUN_EXPORT}")

TRAIN_BASE_IMAGE : gcr.io/deeplearning-platform-release/tf-gpu.2-13.py310
NVTOP_RUN        : RUN apt update && apt -y install nvtop
RUN_EXPORT       : RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/


In [13]:
dockerfile = f'''
FROM {TRAIN_BASE_IMAGE}

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY /requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

RUN ls $APP_HOME

COPY src/reward_factory.py $APP_HOME/src/reward_factory.py
COPY src/train_utils.py $APP_HOME/src/train_utils.py
COPY src/agents $APP_HOME/src/agents
COPY src/networks $APP_HOME/src/networks
COPY src/trainer $APP_HOME/src/trainer
COPY src/data $APP_HOME/src/data

{NVTOP_RUN}

RUN ls $APP_HOME

{RUN_EXPORT}

RUN pip freeze

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.trainer.task"]
'''
print(dockerfile)


FROM gcr.io/deeplearning-platform-release/tf-gpu.2-13.py310

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY /requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

RUN ls $APP_HOME

COPY src/reward_factory.py $APP_HOME/src/reward_factory.py
COPY src/train_utils.py $APP_HOME/src/train_utils.py
COPY src/agents $APP_HOME/src/agents
COPY src/networks $APP_HOME/src/networks
COPY src/trainer $APP_HOME/src/trainer
COPY src/data $APP_HOME/src/data

RUN apt update && apt -y install nvtop

RUN ls $APP_HOME

RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/

RUN pip freeze

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.trainer.task"]



## Write dockerfile

In [14]:
with open(f'{DOCKERNAME_02}', 'w') as f:
    f.write(dockerfile)

## Build image with Cloud Build

Building images with Cloud Build is best practices
* images are centrally stored and better managed for robust CI/CD
* building images on local workbench instance can alter notebook image config (base image for notebooks vs train images are different)
* if building locally, consider using virutal environments

#### set `.gcloudignore`
* to adjust this see the `gcloudignore` section at the end of `00-env-setup.ipynb` notebook

In [15]:
%%writefile .gcloudignore
.gcloudignore
WIP/*
imgs/*
learning/*
*.pkl
*.png
*.ipynb
.git
.github
.gitignore
.DS_Store
*.md
*.tfrecord
.ipynb_checkpoints/*
*cpython-37.pyc
**.cpython-310.pyc
*/__pycache__/*
src/ranking/*
src/archive/*
05-online-learning/*
04-pipelines/*
03-ranking/*
01-baseline-perarm-bandit/*
00-data-prep-eda/*
src/pred/*
Dockerfile_perarm_feats_tpu
Dockerfile_predict_mab_02
vertex_env/*
credentials.json
*/cpr_dir/*
*/local_model_dir/*
src/data_preprocessor/*
src/data/local_data/*

Overwriting .gcloudignore


In [16]:
# check eligble files
# !gcloud meta list-files-for-upload

### Submit container to Cloud Build

In [17]:
# image definitions for training
MACHINE_TYPE            ='e2-highcpu-32'
FILE_LOCATION           = "." # './src'

print(f"DOCKERNAME_02 : {DOCKERNAME_02}")
print(f"IMAGE_URI_02  : {IMAGE_URI_02}")
print(f"FILE_LOCATION : {FILE_LOCATION}")
print(f"MACHINE_TYPE  : {MACHINE_TYPE}")

DOCKERNAME_02 : Dockerfile_perarm_feats
IMAGE_URI_02  : gcr.io/hybrid-vertex/train-perarm-feats-v2
FILE_LOCATION : .
MACHINE_TYPE  : e2-highcpu-32


In [18]:
! gcloud builds submit -q --config ./cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME_02,_IMAGE_URI=$IMAGE_URI_02,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

Creating temporary tarball archive of 35 file(s) totalling 536.0 KiB before compression.
Uploading tarball of [.] to [gs://hybrid-vertex_cloudbuild/source/1710365564.031553-7ff092eb452a47fa8220460bf2437a06.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/hybrid-vertex/locations/global/builds/7cd40914-6674-4fdb-9923-708b9e6d22d2].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/7cd40914-6674-4fdb-9923-708b9e6d22d2?project=934903580331 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "7cd40914-6674-4fdb-9923-708b9e6d22d2"

FETCHSOURCE
Fetching storage object: gs://hybrid-vertex_cloudbuild/source/1710365564.031553-7ff092eb452a47fa8220460bf2437a06.tgz#1710365564705325
Copying gs://hybrid-vertex_cloudbuild/source/1710365564.031553-7ff092eb452a47fa8220460bf2437a06.tgz#1710365564705325...
/ [1 files][122.2 KiB/122.2 KiB]                                                
Operation completed over 1 objects/12

## (Optional) Build Image Locally

Building images with Cloud Build is best practices
* images are centrally stored and better managed for robust CI/CD
* building images on local workbench instance can alter notebook image config (base image for notebooks vs train images are different)
* if building locally, consider using virutal environments

Provide a name for your dockerfile and make sure you are authenticated

In [23]:
# ! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet

In [24]:
print("copy these commands into terminal:\n")
print(f"virtualenv vertex_env")
print(f"source vertex_env/bin/activate")

copy these commands into terminal:

virtualenv vertex_env
source vertex_env/bin/activate


In [26]:
# # set variables if running in terminal
print("copy these commands into terminal:\n")
print(f"export REMOTE_IMAGE_NAME={REMOTE_IMAGE_NAME}")
print(f"export DOCKERNAME={DOCKERNAME}")
print(f"docker build -t $REMOTE_IMAGE_NAME -f ./$DOCKERNAME .")

copy these commands into terminal:

export REMOTE_IMAGE_NAME=us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/train-perarm-feats-v2
export DOCKERNAME=Dockerfile_perarm_feats
docker build -t $REMOTE_IMAGE_NAME -f ./$DOCKERNAME .


In [27]:
# !docker build -t $REMOTE_IMAGE_NAME -f $DOCKERNAME .

### Push container to Registry

In [28]:
# ### push the container to registry

print("copy this command into terminal:\n")
print(f"docker push $REMOTE_IMAGE_NAME")

# !docker push $REMOTE_IMAGE_NAME

copy this command into terminal:

docker push $REMOTE_IMAGE_NAME


### GPU profiling

> enter these commands in the Vertex interactive terminal:

```bash
sudo apt update
sudo apt -y install nvtop
```

## TPU (wip)

> TODO 

* Libtpu [versions](https://cloud.google.com/tpu/docs/supported-tpu-configurations#libtpu_versions)

In [ ]:
# !gsutil ls gs://cloud-tpu-tpuvm-artifacts/tensorflow/tf-2.15.0
# !gsutil ls gs://cloud-tpu-tpuvm-artifacts/tensorflow/tf-2.14.0

In [ ]:
# !gsutil ls gs://cloud-tpu-tpuvm-artifacts/libtpu/1.9.0

In [ ]:
# dockerfile = f'''
# FROM python:3.11

# ENV PYTHONUNBUFFERED True

# RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/libtpu/1.8.0/libtpu.so -O /lib/libtpu.so
# RUN chmod 777 /lib/libtpu.so

# RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/tf-2.14.0/tensorflow-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# RUN pip3 install tensorflow-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# RUN rm tensorflow-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

# ENV APP_HOME /workspace

# WORKDIR $APP_HOME

# COPY /requirements.txt $APP_HOME/requirements.txt

# RUN pip install --upgrade pip
# RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

# RUN ls $APP_HOME

# COPY src/perarm_features $APP_HOME/src/perarm_features
# COPY src/per_arm_rl $APP_HOME/src/per_arm_rl

# RUN ls $APP_HOME

# {RUN_EXPORT}

# RUN pip freeze

# ENV TPU_NAME=local

# # Sets up the entry point to invoke the task.
# ENTRYPOINT ["python3", "-m", "src.perarm_features.task"]
# '''
# print(dockerfile)

**Finished**